In [1]:
act = 365

import pandas as pd
import numpy as np
import scipy.optimize as opt
from numba import jit, njit, set_num_threads
set_num_threads(8)


import pickle
with open("Data/gaussian_cdf.pkl", "rb") as f:
    pkl = pickle.load(f)

Rates_monthly_data = pd.read_csv("Data/rates_monthly.csv", index_col=0, parse_dates=True)


### GLOBAL VARIABLES ###
MTG_MAT = 30
MONTHLY = True
factor = 1
if MONTHLY:
    factor *= 12

TRIANG = np.tri(MTG_MAT*factor+1, MTG_MAT*factor+1).T
ALL_CPNS = np.arange(2.5, 7.0, 0.5)
ALL_CPNS_TBAS = np.arange(1.5, 8.0, 0.5)


OAS_DATA = pd.read_csv("Data/OAS_vs_TREAS.csv", index_col=0)
OAS_DATA.index = pd.to_datetime(OAS_DATA.index, format="%Y-%m-%d")
OAS_DATA = OAS_DATA.iloc[:, :9]
OAS_DATA.columns = ALL_CPNS
OAS_DATA.dropna(how='all', inplace=True)

TBAS_PRICES = pd.read_csv("Data/GENERIC_TBAS.csv", index_col=0).dropna(how="all")
TBAS_PRICES.index = pd.to_datetime(TBAS_PRICES.index, format="%Y-%m-%d")
TBAS_PRICES.columns = ALL_CPNS_TBAS
TBAS_PRICES.dropna(how='all', inplace=True)

IMPLIED_VOL_DATA = pd.read_csv("Data/implied_vol.csv", index_col=0).dropna(how="all")
IMPLIED_VOL_DATA.index = pd.to_datetime(IMPLIED_VOL_DATA.index, format="%Y-%m-%d")
IMPLIED_VOL_DATA = IMPLIED_VOL_DATA['3M10Y'].dropna()
IMPLIED_VOL_DATA = IMPLIED_VOL_DATA.apply(lambda x: float(x.replace(',', '.'))).sort_index()
IMPLIED_VOL_DATA.dropna(how='all', inplace=True)
implied_usd = IMPLIED_VOL_DATA


MTG = pd.read_csv('Data/MTG_RATE.csv', index_col=0)
MTG.index = pd.to_datetime(MTG.index, format="%m/%d/%y")
MTG = MTG[['Mortgage Rate']]
MTG.dropna(how='all', inplace=True)
for i, val in enumerate(ALL_CPNS):
    MTG[val] = val - MTG["Mortgage Rate"]

LOAN_AGE = pd.read_csv('Data/loan_age.csv', index_col=0)
LOAN_AGE.index = pd.to_datetime(LOAN_AGE.index, format="%m/%d/%y")
LOAN_AGE = LOAN_AGE[[str(c) for c in ALL_CPNS]]
LOAN_AGE.columns = ALL_CPNS
LOAN_AGE.dropna(how='all', inplace=True)

dates = [ d for d in Rates_monthly_data.index if 
        (
            d in IMPLIED_VOL_DATA.index and 
            d in TBAS_PRICES.index and 
            d in MTG.index and 
            d in LOAN_AGE.index
        )]

Rates_monthly_data= Rates_monthly_data         .loc[dates]
IMPLIED_VOL_DATA  = IMPLIED_VOL_DATA  .loc[dates]
TBAS_PRICES       = TBAS_PRICES       .loc[dates] 
MTG               = MTG               .loc[dates] 
LOAN_AGE          = LOAN_AGE          .loc[dates]

In [2]:
# fixeds params
LOW_CPR = 0.1
HIGH_CPR = 0.4

In [3]:
# Optimized function

def retrieve_erf(x, my_pickle):
    """Retrieve the Gaussian CDF of x efficiently."""
    x_values, cdf_values = my_pickle
    closest_x_idx = np.argmin(np.abs(x_values - x))
    return cdf_values[closest_x_idx]


def PRICE_MBS(SOFR_DATA, COUPON, OAS, 
              MTG_MATURITY=30,
              VOL=100,
              floor_quantity=0.5, most_expensive_quantity=0.5,
              LOW_CPR=LOW_CPR, HIGH_CPR=HIGH_CPR, 
              my_pickle=pkl):
    """Compute the price of an MBS with optimized performance."""
    factor = 12
    LENGTH = MTG_MATURITY * factor + 1
    LENGTH_IDX = np.arange(LENGTH)

    COUPON = COUPON/100
    CPN = COUPON/ factor
    
    VOL = VOL*1e-4
    N0 = 100

    # Compute annuity and prepayment schedules
    annuity = N0 * (1 - (1 + CPN) ** -((MTG_MATURITY * factor - LENGTH_IDX))) / (1 - (1 + CPN) ** (-MTG_MATURITY * factor))
    low_cpr = annuity * (1 - LOW_CPR) ** (LENGTH_IDX / factor)
    #high_cpr = low_cpr * (1 - HIGH_CPR) / (1 - LOW_CPR)
    high_cpr = annuity * (1 - HIGH_CPR)** (LENGTH_IDX / factor)

    # Compute cash flows
    cf_bond = np.zeros(LENGTH)
    cf_bond[1:] = (low_cpr[:-1]) * (1 + CPN) - low_cpr[1:]
    
    # Discount factors
    rates_oas = (SOFR_DATA + OAS/100)[:LENGTH] / 100
    ti_in_years = LENGTH_IDX * 30 / 365
    zero_coupon = np.exp(-rates_oas * ti_in_years)
    bond_price = np.sum(cf_bond * zero_coupon)

    # Option pricing calculations
    notional_diff = low_cpr - high_cpr
    weights = np.zeros(LENGTH)
    weights[1:] = zero_coupon[1:] * low_cpr[:-1]

    duration = np.cumsum(weights[::-1])[::-1]
    duration = np.roll(duration, -1)
    duration[:-1] = duration[:-1] / (zero_coupon * low_cpr * factor)[:-1]
    duration[-1] = 0

    fwd_rates = np.zeros(LENGTH)
    fwd_rates[1:] = (zero_coupon[:-1] / zero_coupon[1:] - 1) * factor
    weighted_fwd_rates = weights * fwd_rates

    swap_rate = np.cumsum(weighted_fwd_rates[::-1])[::-1]
    swap_rate = np.roll(swap_rate, -1)
    swap_rate[:-1] = swap_rate[:-1] / (duration * (zero_coupon * low_cpr * factor))[:-1]
    swap_rate[-1] = 0

    x_swaption = (COUPON - swap_rate)[1:-1] / (VOL * np.sqrt(LENGTH_IDX / factor))[1:-1]
    ERF_swpt = np.array([retrieve_erf(x, my_pickle) for x in x_swaption])
    density_swpt = (1 / np.sqrt(2 * np.pi)) * np.exp((-x_swaption**2) / 2)
    p_swaptions = (duration * VOL * np.sqrt(LENGTH_IDX / factor))[1:-1] * (x_swaption * ERF_swpt + density_swpt)
    p_swaptions = notional_diff[1:-1] * zero_coupon[1:-1] * p_swaptions
    ret_max_spt = np.nanmax(p_swaptions)

    x_cap = (COUPON - fwd_rates)[1:-1] / (VOL * np.sqrt(LENGTH_IDX / factor))[1:-1]
    ERF_cap = np.array([retrieve_erf(x, my_pickle) for x in x_cap])
    density_cap = (1 / np.sqrt(2 * np.pi)) * np.exp((-x_cap**2) / 2)
    p_cap = (zero_coupon * VOL * np.sqrt(LENGTH_IDX / factor))[1:-1] * (x_cap * ERF_cap + density_cap) * notional_diff[1:-1] / factor
    caplet_sum = np.nansum(p_cap)

    return bond_price - (most_expensive_quantity * ret_max_spt + floor_quantity * caplet_sum)


In [20]:
Rates_monthly_data_array = Rates_monthly_data.to_numpy()
loan_age_array = LOAN_AGE[ALL_CPNS].to_numpy()
tba_price_array = TBAS_PRICES[ALL_CPNS].to_numpy()

n_dates = len(dates)
n_cpn = len(ALL_CPNS) 

## sanity checks 

from tqdm import tqdm


date_idx = 0 
cpn_idx = 0

rates_t = Rates_monthly_data_array[date_idx]


l = np.zeros(n_cpn)
for _ in range(10):
    for cpn_idx in range(n_cpn):
        market_price = tba_price_array[date_idx, cpn_idx]            
        MTG_MATURITY = round(30 - loan_age_array[date_idx, cpn_idx])
        c = ALL_CPNS[cpn_idx]
        l[cpn_idx] = PRICE_MBS(rates_t, c, 0, MTG_MATURITY)
l

array([100.90883927, 103.05405008, 105.00083278, 106.85520713,
       108.64709483, 110.34943176, 111.97709243, 113.541538  ,
       115.1636467 ])

array([30., 29., 29., 29., 28., 27., 26., 25., 26.])

In [26]:
SOFR_DATA = rates_t
COUPON = ALL_CPNS
OAS = np.zeros_like(ALL_CPNS)
MTG_MATURITY=(30 - loan_age_array[date_idx]).round()
VOL=100
floor_quantity=0.5
most_expensive_quantity=0.5
LOW_CPR=LOW_CPR*np.ones_like(ALL_CPNS)
HIGH_CPR=HIGH_CPR*np.ones_like(ALL_CPNS) 
my_pickle=pkl

In [32]:
"""Compute the price of an MBS with optimized performance."""
factor = 12
LENGTH = MTG_MATURITY * factor + 1

In [38]:
max_length = LENGTH.max()
LENGTH_IDX = np.tile(np.arange(max_length), len(LENGTH), axis=1)
LENGTH_IDX.shape

TypeError: tile() got an unexpected keyword argument 'axis'

In [ ]:




LENGTH_IDX = np.arange(LENGTH)

COUPON = COUPON/100
CPN = COUPON/ factor

VOL = VOL*1e-4
N0 = 100

# Compute annuity and prepayment schedules
annuity = N0 * (1 - (1 + CPN) ** -((MTG_MATURITY * factor - LENGTH_IDX))) / (1 - (1 + CPN) ** (-MTG_MATURITY * factor))
low_cpr = annuity * (1 - LOW_CPR) ** (LENGTH_IDX / factor)
#high_cpr = low_cpr * (1 - HIGH_CPR) / (1 - LOW_CPR)
high_cpr = annuity * (1 - HIGH_CPR)** (LENGTH_IDX / factor)

# Compute cash flows
cf_bond = np.zeros(LENGTH)
cf_bond[1:] = (low_cpr[:-1]) * (1 + CPN) - low_cpr[1:]

# Discount factors
rates_oas = (SOFR_DATA + OAS/100)[:LENGTH] / 100
ti_in_years = LENGTH_IDX * 30 / 365
zero_coupon = np.exp(-rates_oas * ti_in_years)
bond_price = np.sum(cf_bond * zero_coupon)

# Option pricing calculations
notional_diff = low_cpr - high_cpr
weights = np.zeros(LENGTH)
weights[1:] = zero_coupon[1:] * low_cpr[:-1]

duration = np.cumsum(weights[::-1])[::-1]
duration = np.roll(duration, -1)
duration[:-1] = duration[:-1] / (zero_coupon * low_cpr * factor)[:-1]
duration[-1] = 0

fwd_rates = np.zeros(LENGTH)
fwd_rates[1:] = (zero_coupon[:-1] / zero_coupon[1:] - 1) * factor
weighted_fwd_rates = weights * fwd_rates

swap_rate = np.cumsum(weighted_fwd_rates[::-1])[::-1]
swap_rate = np.roll(swap_rate, -1)
swap_rate[:-1] = swap_rate[:-1] / (duration * (zero_coupon * low_cpr * factor))[:-1]
swap_rate[-1] = 0

x_swaption = (COUPON - swap_rate)[1:-1] / (VOL * np.sqrt(LENGTH_IDX / factor))[1:-1]
ERF_swpt = np.array([retrieve_erf(x, my_pickle) for x in x_swaption])
density_swpt = (1 / np.sqrt(2 * np.pi)) * np.exp((-x_swaption**2) / 2)
p_swaptions = (duration * VOL * np.sqrt(LENGTH_IDX / factor))[1:-1] * (x_swaption * ERF_swpt + density_swpt)
p_swaptions = notional_diff[1:-1] * zero_coupon[1:-1] * p_swaptions
ret_max_spt = np.nanmax(p_swaptions)

x_cap = (COUPON - fwd_rates)[1:-1] / (VOL * np.sqrt(LENGTH_IDX / factor))[1:-1]
ERF_cap = np.array([retrieve_erf(x, my_pickle) for x in x_cap])
density_cap = (1 / np.sqrt(2 * np.pi)) * np.exp((-x_cap**2) / 2)
p_cap = (zero_coupon * VOL * np.sqrt(LENGTH_IDX / factor))[1:-1] * (x_cap * ERF_cap + density_cap) * notional_diff[1:-1] / factor
caplet_sum = np.nansum(p_cap)

bond_price - (most_expensive_quantity * ret_max_spt + floor_quantity * caplet_sum)
